# Container DAG


https://docker-py.readthedocs.io/en/stable/index.html

https://docs.docker.com/reference/cli/docker/

In [1]:
from abc import ABCMeta, abstractmethod
import json
import random
import subprocess
import sys

from docker import from_env

In [2]:
sys.path.append("../../src")

from dockerweed.topological_sort import kahns_algorithm
from dockerweed.process import *
from dockerweed.container import *

## Open Docker client

In [3]:
client = from_env()

### Get container processes

In [4]:
containers = []
processes = {}
# Loop through images. 
# Exclude intermediate layer images (all=False) and dangling images.
for image in client.images.list(all=False, filters={"dangling":False}):
    
    # Image tag
    tags = image.tags
    if not tags:
        continue
    image_tag = tags[0]
    
    # Labels with node names and commands.
    labels = image.attrs['ContainerConfig']['Labels']
    if labels is None:
        continue
    process_cmd = {name: cmd for name, cmd in labels.items() if name.startswith("process.")}
    if not process_cmd:
        continue
        
    # Processes
    for name, cmd in process_cmd.items():
        # Container should have no entry point, or this will call the entrypoint rather than label command.
        result = client.containers.run(image=image_tag, command=cmd, remove=True, stdout=True)
        try:
            dic = json.loads(result)
            if "description" in dic and "outputs" in dic and "inputs" in dic:
                
                if not containers or containers[-1].image != image_tag:
                    container = NodeContainer(image_tag=tags[0])
                    containers.append(container) 
                
                processes[name] = ContainerProcess(
                    name = name,
                    description = dic["description"],
                    container = containers[-1].name,
                    command = cmd,
                    inputs = dic["inputs"],
                    outputs = dic["outputs"]
                )    
        except json.decoder.JSONDecodeError:
            continue
    
print([container.name for container in containers])
print([process.name for process in processes.values()])

['multiple_divide', 'add_subtract']
['process.c_divide', 'process.c_multiply', 'process.python_add', 'process.python_subtract']


### Start containers

In [5]:
for container in containers:
    container.start()

3382aa9b740e7c04717147187ea5a44f28c7021ebe661921bb7b4a5ffb38f774
cfc66388f3560b0ffaa79fd62f2dff0db7665dc78d59f65b3aefc7b11d716a7d


### Add generator processes

In [6]:
processes["random float"] = UniformFloatProcess()
processes["random choice"] = RandomChoice()

### Information about processes

In [7]:
for i, process in enumerate(processes.values()):
    print("-"*30)
    print(f"#{i} {process.name}")
    print(f"\t{process.description}")
    print(f"\tinputs = {process.inputs}")
    print(f"\toutputs = {process.outputs}")

------------------------------
#0 process.c_divide
	Division. z = a / b
	inputs = {'a': 1.0, 'b': 1.0}
	outputs = {'z': 1.0}
------------------------------
#1 process.c_multiply
	Multiplication. z = a * b
	inputs = {'a': 1.0, 'b': 1.0}
	outputs = {'z': 1.0}
------------------------------
#2 process.python_add
	Addition. z = x + y.
	inputs = {'x': 1.0, 'y': 1.0}
	outputs = {'z': 1.0}
------------------------------
#3 process.python_subtract
	Addition. z = x + y.
	inputs = {'x': 1.0, 'y': 1.0}
	outputs = {'z': 1.0}
------------------------------
#4 random float
	A uniformly distributed random float.
	inputs = {'min': 0.0, 'max': 1.0}
	outputs = {'n': 0.5}
------------------------------
#5 random choice
	A rnadom choice.
	inputs = {'choices': [1, 2, 3, 4]}
	outputs = {'choice': 1}


### Run individual processes

In [8]:
processes["process.python_add"].run({'x': 2.34, 'y': 10.78})

{'z': 13.12}

In [9]:
processes["process.c_multiply"].run({'a': 89.987, 'b': 10.0})

{'x': 899.8699951171875}

In [10]:
for _ in range(10):
    x = processes["random choice"].run({'choices': [3, 4, 8]})
    print(x)

{'choice': 3}
{'choice': 8}
{'choice': 8}
{'choice': 3}
{'choice': 3}
{'choice': 8}
{'choice': 3}
{'choice': 3}
{'choice': 3}
{'choice': 4}


### Run a graph of processes

#### Specify the graph

In [11]:
graph_specification = {
    "node alpha": {
        "process": "random choice",
        "inputs": {'choices': [1.2, 5.4, 6.7]}
    },
    "node beta": {
        "process": "random float",
        "inputs": {"min": -5.0, "max": 5.0}
    },
    "node a": {
        "process": "process.python_add",
        "inputs": {'x': ("node alpha", "choice"), 'y': 16.5}
    },
    "node b": {
        "process": "process.python_add",
        "inputs": {'x': ("node a", "z"), 'y': ("node beta", "n")}
    },
    "node c": {
        "process": "process.c_divide",
        "inputs": {'a': ("node a", "z"), 'b': ("node b", "z")}
    },
}

#### Get the topological (execution) order of the nodes

In [12]:
graph_topology = {}
    
for node, content in graph_specification.items():
    if "inputs" not in content or "process" not in content or not isinstance(content["inputs"], dict):
        continue
        
    graph_topology[node] = [
        value[0]
        for input_name, value in content["inputs"].items()
        if isinstance(value, tuple)
    ]
    
graph_topology

{'node alpha': [],
 'node beta': [],
 'node a': ['node alpha'],
 'node b': ['node a', 'node beta'],
 'node c': ['node a', 'node b']}

In [13]:
topological_order = kahns_algorithm(graph_topology, incoming=True)
topological_order

['node alpha', 'node beta', 'node a', 'node b', 'node c']

#### Execute the graph

In [15]:
inputs = {}
outputs = {}

for node in topological_order:
    
    # Find node's process.
    process_name = graph_specification[node]["process"]
    if process_name not in processes:
        raise KeyError(f"No process called {process_name}.")
    process_obj = processes[process_name]

    # Substitute tuple inputs with outputs from prior nodes.
    inputs[node] = graph_specification[node]["inputs"].copy()
    for input_name, value in inputs[node].items():
        if not isinstance(value, tuple):
            continue
        try:
            incoming_node = value[0]
            incoming_variable = value[1]
            incoming_value = outputs[incoming_node][incoming_variable]
            inputs[node][input_name] = incoming_value
        except KeyError:
            raise KeyError(f"Invalid inputs for")
        
    # Execute, storing the outputs.
    outputs[node] = process_obj.run(inputs[node])
    
print("-"*20)
print(inputs)
print("-"*20)
print(outputs)

--------------------
{'node alpha': {'choices': [1.2, 5.4, 6.7]}, 'node beta': {'min': -5.0, 'max': 5.0}, 'node a': {'x': 1.2, 'y': 16.5}, 'node b': {'x': 17.7, 'y': 2.2129656701057074}, 'node c': {'a': 17.7, 'b': 19.912965670105706}}
--------------------
{'node alpha': {'choice': 1.2}, 'node beta': {'n': 2.2129656701057074}, 'node a': {'z': 17.7}, 'node b': {'z': 19.912965670105706}, 'node c': {'x': 0.8888681530952454}}


### Stop containers

In [41]:
for container in containers:
    container.stop()

multiple_divide
add_subtract


In [ ]:
# echo '{"x": 11.98769, "y": 186.78}' | docker exec -i some-node-a python run.py


## Close client

In [42]:
client.close()